In [ ]:
import pandas as pd

# Load the dataset
file_path = "hp24_hr.csv"  # Replace with your actual file path
df = pd.read_csv(file_path)

# Convert the time column to datetime format and remove timezone information
df['time'] = pd.to_datetime(df['time']).dt.strftime('%Y-%m-%d %H:%M:%S')

# Sort the dataframe by time in increasing order
df_sorted = df.sort_values(by='time')

# Display the first few rows of the sorted dataframe
print(df_sorted.head())

In [ ]:
import pandas as pd

# Load the steps dataset
file_path_steps = "hp24_steps.csv"  # Replace with your actual file path
df_steps = pd.read_csv(file_path_steps)

# Convert the time column to datetime format and remove timezone information
df_steps['time'] = pd.to_datetime(df_steps['time']).dt.strftime('%Y-%m-%d %H:%M:%S')

# Sort the dataframe by time in increasing order
df_steps_sorted = df_steps.sort_values(by='time')

# Display the first few rows of the sorted dataframe
print(df_steps_sorted.head())

In [ ]:
# Load the blood pressure dataset
file_path_bp = "blood_pressure_readings_ID24_cleaned.csv"  # Replace with your actual file path
df_bp = pd.read_csv(file_path_bp)

# Select relevant columns
df_bp = df_bp[['datetime_local', 'systolic', 'diastolic']]

# Convert the datetime column to datetime format
df_bp['datetime_local'] = pd.to_datetime(df_bp['datetime_local'])

# Sort the dataframe by datetime in increasing order
df_bp_sorted = df_bp.sort_values(by='datetime_local')

# Add a binary classification column for BP spikes
df_bp_sorted['BP_spike'] = ((df_bp_sorted['systolic'] > 130) | (df_bp_sorted['diastolic'] > 80)).astype(int)

# Count the number of BP spikes and total records
total_records = len(df_bp_sorted)
bp_spike_count = df_bp_sorted['BP_spike'].sum()

# Print summary
print(f"Total records: {total_records}")
print(f"Number of BP spikes: {bp_spike_count}")

# Display the first few rows of the processed dataframe
print(df_bp_sorted.head())

In [ ]:
# Load the stress data dataset
file_path_stress = "questionnaire_responses_ID24.csv"  # Replace with your actual file path
df_stress = pd.read_csv(file_path_stress)

# Select relevant columns
df_stress = df_stress[['local_created_at', 'stressLevel_value']]

# Convert the time column to datetime format
df_stress['local_created_at'] = pd.to_datetime(df_stress['local_created_at'])

# Sort the dataframe by time in increasing order
df_stress_sorted = df_stress.sort_values(by='local_created_at')

# Display the first few rows of the processed dataframe
print(df_stress_sorted.head())

In [ ]:
import pandas as pd
import numpy as np

# 📌 **Step 1: Load Datasets**
file_path_hr = "hp24_hr.csv"
file_path_steps = "hp24_steps.csv"
file_path_bp = "blood_pressure_readings_ID24_cleaned.csv"
file_path_stress = "questionnaire_responses_ID24.csv"

df_hr = pd.read_csv(file_path_hr)
df_steps = pd.read_csv(file_path_steps)
df_bp = pd.read_csv(file_path_bp)
df_stress = pd.read_csv(file_path_stress)

# 📌 **Step 2: Convert timestamps to uniform `datetime64[ns]`**
df_hr['time'] = pd.to_datetime(df_hr['time']).dt.tz_localize(None)
df_steps['time'] = pd.to_datetime(df_steps['time']).dt.tz_localize(None)
df_bp['datetime_local'] = pd.to_datetime(df_bp['datetime_local']).dt.tz_localize(None)
df_stress['local_created_at'] = pd.to_datetime(df_stress['local_created_at']).dt.tz_localize(None)

# 📌 **Step 3: Remove Data from `2024-10-23` for BP & Stress**
df_bp = df_bp[df_bp['datetime_local'].dt.date > pd.to_datetime("2024-09-03").date()]
df_stress = df_stress[df_stress['local_created_at'].dt.date > pd.to_datetime("2024-09-03").date()]

# 📌 **Step 4: Sort datasets by time**
df_hr_sorted = df_hr.sort_values(by='time')
df_steps_sorted = df_steps.sort_values(by='time')
df_bp_sorted = df_bp.sort_values(by='datetime_local')
df_stress_sorted = df_stress.sort_values(by='local_created_at')

# 📌 **Step 5: Add BP spike classification and Tertile Classification**
# Retain the threshold-based definition for reference:
df_bp_sorted['BP_spike_threshold'] = ((df_bp_sorted['systolic'] > 130) | (df_bp_sorted['diastolic'] > 80)).astype(int)

# New binary definition based on mean values:
mean_systolic = df_bp_sorted['systolic'].mean()
mean_diastolic = df_bp_sorted['diastolic'].mean()
df_bp_sorted['BP_spike_mean'] = ((df_bp_sorted['systolic'] > mean_systolic) | (df_bp_sorted['diastolic'] > mean_diastolic)).astype(int)

# Tertile classification for systolic and diastolic:
systolic_lower = df_bp_sorted['systolic'].quantile(0.333)
systolic_upper = df_bp_sorted['systolic'].quantile(0.667)
diastolic_lower = df_bp_sorted['diastolic'].quantile(0.333)
diastolic_upper = df_bp_sorted['diastolic'].quantile(0.667)

def classify_bp(value, lower, upper):
    if value < lower:
        return "low"
    elif value < upper:
        return "mid"
    else:
        return "high"

df_bp_sorted['systolic_tertile'] = df_bp_sorted['systolic'].apply(lambda x: classify_bp(x, systolic_lower, systolic_upper))
df_bp_sorted['diastolic_tertile'] = df_bp_sorted['diastolic'].apply(lambda x: classify_bp(x, diastolic_lower, diastolic_upper))

# 📌 **Step 6: Merge HR & Steps Data Based on Nearest Timestamp**
df_biosignals = pd.merge_asof(df_hr_sorted, df_steps_sorted, on='time', direction='backward', suffixes=('_hr', '_steps'))

# 📌 **Step 7: Compute Rolling Window Statistics for HR & Steps**
df_biosignals.set_index('time', inplace=True)
time_windows = [5, 10, 30, 60]  # Define time windows (minutes)
for window in time_windows:
    window_str = f"{window}min"
    # Rolling windows here include the current row; if you need strictly previous data, consider shifting afterward.
    df_biosignals[f'hr_mean_{window_str}'] = df_biosignals['value_hr'].rolling(f"{window}min").mean()
    df_biosignals[f'hr_min_{window_str}'] = df_biosignals['value_hr'].rolling(f"{window}min").min()
    df_biosignals[f'hr_max_{window_str}'] = df_biosignals['value_hr'].rolling(f"{window}min").max()
    df_biosignals[f'hr_std_{window_str}'] = df_biosignals['value_hr'].rolling(f"{window}min").std()
    df_biosignals[f'steps_total_{window_str}'] = df_biosignals['value_steps'].rolling(f"{window}min").sum()
    df_biosignals[f'steps_mean_{window_str}'] = df_biosignals['value_steps'].rolling(f"{window}min").mean()
    df_biosignals[f'steps_min_{window_str}'] = df_biosignals['value_steps'].rolling(f"{window}min").min()
    df_biosignals[f'steps_max_{window_str}'] = df_biosignals['value_steps'].rolling(f"{window}min").max()
    df_biosignals[f'steps_std_{window_str}'] = df_biosignals['value_steps'].rolling(f"{window}min").std()
    df_biosignals[f'steps_diff_{window_str}'] = df_biosignals[f'steps_max_{window_str}'] - df_biosignals[f'steps_min_{window_str}']
# Reset index after rolling computation
df_biosignals.reset_index(inplace=True)

# 📌 **Step 8: Merge BP Data with HR & Steps Features**
df_merged = pd.merge_asof(df_bp_sorted, df_biosignals, left_on='datetime_local', right_on='time', direction='backward')

# 📌 **Step 9: Incorporate Stress Data (±15 minutes window)**
def extract_stress_features(bp_time, df_stress):
    start_time = bp_time - pd.Timedelta(minutes=15)
    end_time = bp_time + pd.Timedelta(minutes=15)
    stress_values = df_stress[(df_stress['local_created_at'] >= start_time) &
                              (df_stress['local_created_at'] <= end_time)]['stressLevel_value']
    return pd.Series({
        'stress_mean': stress_values.mean(),
        'stress_min': stress_values.min(),
        'stress_max': stress_values.max(),
        'stress_std': stress_values.std()
    })

df_stress_features = df_bp_sorted['datetime_local'].apply(lambda x: extract_stress_features(x, df_stress_sorted))
df_merged = pd.concat([df_merged, df_stress_features], axis=1)

# 📌 **Step 10: Create Additional Engineered Features**

# ✅ Lagged Features: only using previous data (based on BP_spike_mean and other past features)
lag_features = ['stress_mean', 'BP_spike_mean', 'hr_mean_5min', 'steps_total_10min']
for feature in lag_features:
    for lag in [1, 3, 5]:
        df_merged[f'{feature}_lag_{lag}'] = df_merged[feature].shift(lag)

# ✅ Feature Interactions
df_merged['hr_steps_ratio'] = df_merged['hr_mean_5min'] / (df_merged['steps_total_10min'] + 1)
df_merged['stress_weighted_hr'] = df_merged['hr_mean_5min'] * df_merged['stress_mean']
df_merged['stress_steps_ratio'] = df_merged['stress_mean'] / (df_merged['steps_total_10min'] + 1)
df_merged['steps_hr_variability_ratio'] = df_merged['steps_std_10min'] / (df_merged['hr_std_10min'] + 1e-5)

# ✅ Rolling Aggregations
df_merged['hr_mean_rolling_3'] = df_merged['hr_mean_5min'].rolling(3).mean()
df_merged['steps_total_rolling_5'] = df_merged['steps_total_10min'].rolling(5).mean()
df_merged['hr_std_rolling_3'] = df_merged['hr_std_10min'].rolling(3).std()
df_merged['cumulative_stress_30min'] = df_merged['stress_mean'].rolling(3).sum()
df_merged['cumulative_steps_30min'] = df_merged['steps_total_10min'].rolling(3).sum()

# ✅ Contextual Features
df_merged['hour_of_day'] = df_merged['datetime_local'].dt.hour
df_merged['day_of_week'] = df_merged['datetime_local'].dt.dayofweek
df_merged['is_working_hours'] = df_merged['hour_of_day'].between(9, 17).astype(int)
df_merged['is_weekend'] = (df_merged['day_of_week'] >= 5).astype(int)

# ✅ Time Since Last BP Spike (based on BP_spike_mean)
# Use previous row's timestamp (if it was a spike) by shifting the datetime column.
df_merged['last_spike_time'] = df_merged['datetime_local'].shift(1).where(df_merged['BP_spike_mean'].shift(1) == 1)
df_merged['last_spike_time'] = df_merged['last_spike_time'].ffill()
df_merged['time_since_last_BP_spike'] = (df_merged['datetime_local'] - df_merged['last_spike_time']).dt.total_seconds() / 60
df_merged.drop(columns=['last_spike_time'], inplace=True)

# ✅ Drop Direct Current Row Measurements to Prevent Data Leakage
# We drop the raw systolic and diastolic values and the threshold-based spike,
# but now keep the tertile classifications for use in the 3-class problem.
drop_cols = ['systolic', 'diastolic', 'BP_spike_threshold']
df_merged.drop(columns=drop_cols, inplace=True)

# 📌 **Step 11: Handle Missing Values**
df_merged.ffill(inplace=True)
df_merged.bfill(inplace=True)  # Fixes rolling feature NaNs at the beginning

# 📌 **Step 12: Save Processed Dataset**
df_merged.to_csv("processed_bp_prediction_data_multiclass.csv", index=False)
print("✅ Final dataset saved as 'processed_bp_prediction_data_multiclass.csv'.")
print(df_merged.head())


In [1]:
import time
start_time = time.time()

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Bidirectional, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import ADASYN
import kerastuner as kt

#############################################
# Custom Attention Layers
#############################################
class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight",
                                 shape=(input_shape[-1], 1),
                                 initializer="normal",
                                 trainable=True)
        self.b = self.add_weight(name="att_bias",
                                 shape=(input_shape[1], 1),
                                 initializer="zeros",
                                 trainable=True)
        super(AttentionLayer, self).build(input_shape)
    def call(self, x):
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.backend.softmax(e, axis=1)
        output = x * a
        return tf.keras.backend.sum(output, axis=1)

class MultiHeadAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, num_heads, key_dim, **kwargs):
        super(MultiHeadAttentionLayer, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.key_dim)
    def call(self, inputs):
        attn_output = self.mha(query=inputs, key=inputs, value=inputs)
        return tf.reduce_mean(attn_output, axis=1)

class SelfAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(SelfAttentionLayer, self).__init__(**kwargs)
        self.att = tf.keras.layers.Attention()
    def call(self, x):
        att_out = self.att([x, x])
        return tf.reduce_mean(att_out, axis=1)

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, num_heads, key_dim, ff_dim, rate=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.ff_dim = ff_dim
        self.rate = rate
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
    def build(self, input_shape):
        self.embed_dim = input_shape[-1]
        self.ffn = tf.keras.Sequential([
            Dense(self.ff_dim, activation='relu'),
            Dense(self.embed_dim)
        ])
        super(TransformerBlock, self).build(input_shape)
    def call(self, inputs, training=False):
        attn_output = self.mha(query=inputs, key=inputs, value=inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

#############################################
# 1. Load Processed Dataset and Prepare Targets
#############################################
# Processed CSV must include the tertile columns computed earlier: "systolic_tertile" and "diastolic_tertile".
df = pd.read_csv("processed_bp_prediction_data_multiclass.csv")

# Define features (ensure target columns are excluded)
features = [
    'hr_mean_5min', 'hr_min_5min', 'hr_max_5min', 'hr_std_5min',
    'steps_total_5min', 'steps_mean_5min', 'steps_min_5min', 'steps_max_5min', 'steps_std_5min', 'steps_diff_5min',
    'hr_mean_10min', 'hr_min_10min', 'hr_max_10min', 'hr_std_10min',
    'steps_total_10min', 'steps_mean_10min', 'steps_min_10min', 'steps_max_10min', 'steps_std_10min', 'steps_diff_10min',
    'hr_mean_30min', 'hr_min_30min', 'hr_max_30min', 'hr_std_30min',
    'steps_total_30min', 'steps_mean_30min', 'steps_min_30min', 'steps_max_30min', 'steps_std_30min', 'steps_diff_30min',
    'hr_mean_60min', 'hr_min_60min', 'hr_max_60min', 'hr_std_60min',
    'steps_total_60min', 'steps_mean_60min', 'steps_min_60min', 'steps_max_60min', 'steps_std_60min', 'steps_diff_60min',
    'stress_mean', 'stress_min', 'stress_max', 'stress_std',
    'stress_mean_lag_1', 'stress_mean_lag_3', 'stress_mean_lag_5',
    'BP_spike_mean_lag_1', 'BP_spike_mean_lag_3', 'BP_spike_mean_lag_5',
    'hr_mean_5min_lag_1', 'hr_mean_5min_lag_3', 'hr_mean_5min_lag_5',
    'steps_total_10min_lag_1', 'steps_total_10min_lag_3', 'steps_total_10min_lag_5',
    'hr_steps_ratio', 'stress_weighted_hr', 'stress_steps_ratio', 'steps_hr_variability_ratio',
    'hr_mean_rolling_3', 'steps_total_rolling_5', 'hr_std_rolling_3',
    'cumulative_stress_30min', 'cumulative_steps_30min',
    'hour_of_day', 'day_of_week', 'is_working_hours', 'is_weekend',
    'time_since_last_BP_spike'
]

# Our targets: the tertile classifications for systolic and diastolic values
target_systolic = "systolic_tertile"
target_diastolic = "diastolic_tertile"

# Retain only needed columns
df = df[["datetime_local"] + features + [target_systolic, target_diastolic]]
df[features] = df[features].apply(pd.to_numeric, errors='coerce')
df['datetime_local'] = pd.to_datetime(df['datetime_local'])

# Split train and test by time
train_cutoff = df['datetime_local'].min() + pd.Timedelta(days=20)
train_data = df[df['datetime_local'] < train_cutoff]
test_data = df[df['datetime_local'] >= train_cutoff]

X_train = train_data[features].copy()
X_test = test_data[features].copy()

# One-hot encode targets for multi-class classification
y_train_systolic = pd.get_dummies(train_data[target_systolic])
y_train_diastolic = pd.get_dummies(train_data[target_diastolic])
y_test_systolic = pd.get_dummies(test_data[target_systolic])
y_test_diastolic = pd.get_dummies(test_data[target_diastolic])

# Ensure consistent ordering of columns
order = ['low', 'mid', 'high']
y_train_systolic = y_train_systolic[order]
y_train_diastolic = y_train_diastolic[order]
y_test_systolic = y_test_systolic[order]
y_test_diastolic = y_test_diastolic[order]

# Create dictionary targets for the multi-output model
y_train_multi = {'systolic': y_train_systolic.values, 'diastolic': y_train_diastolic.values}
y_test_multi = {'systolic': y_test_systolic.values, 'diastolic': y_test_diastolic.values}

#############################################
# 2. Scaling and Reshaping for LSTM
#############################################
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

#############################################
# 3. Build Multi-Output (Multi-Task) LSTM Model with Keras Tuner
#############################################
def build_model(hp):
    inputs = tf.keras.Input(shape=(X_train_lstm.shape[1], 1))
    
    # Shared LSTM layers
    lstm_units = hp.Int('lstm_units', min_value=64, max_value=256, step=32)
    x = Bidirectional(LSTM(lstm_units, return_sequences=True))(inputs)
    x = BatchNormalization()(x)
    dropout_rate = hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)
    x = Dropout(dropout_rate)(x)
    
    lstm_units_2 = hp.Int('lstm_units_2', min_value=32, max_value=128, step=16)
    x = LSTM(lstm_units_2, return_sequences=True)(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    
    # Attention variant selection
    att_variant = hp.Choice('attention_variant', ['custom', 'multihead', 'selfattention', 'transformer'])
    if att_variant == 'custom':
        att_out = AttentionLayer()(x)
    elif att_variant == 'multihead':
        num_heads = hp.Int('num_heads', min_value=1, max_value=4, step=1)
        key_dim = hp.Int('key_dim', min_value=16, max_value=64, step=16)
        att_out = MultiHeadAttentionLayer(num_heads=num_heads, key_dim=key_dim)(x)
    elif att_variant == 'selfattention':
        att_out = SelfAttentionLayer()(x)
    else:  # transformer variant
        trans_num_heads = hp.Int('trans_num_heads', min_value=1, max_value=4, step=1)
        trans_key_dim = hp.Int('trans_key_dim', min_value=16, max_value=64, step=16)
        ff_dim = hp.Int('ff_dim', min_value=32, max_value=128, step=32)
        transformer_block = TransformerBlock(num_heads=trans_num_heads, key_dim=trans_key_dim, ff_dim=ff_dim, rate=dropout_rate)
        att_out = transformer_block(x)
        att_out = GlobalAveragePooling1D()(att_out)
    
    # Optional shared dense layer
    shared_dense_units = hp.Int('shared_dense_units', min_value=32, max_value=128, step=32)
    shared = Dense(shared_dense_units, activation='relu')(att_out)
    shared = Dropout(dropout_rate)(shared)
    
    # Systolic branch
    systolic_dense_units = hp.Int('systolic_dense_units', min_value=16, max_value=64, step=16)
    systolic_branch = Dense(systolic_dense_units, activation='relu')(shared)
    systolic_branch = Dropout(dropout_rate)(systolic_branch)
    systolic_output = Dense(3, activation='softmax', name='systolic')(systolic_branch)
    
    # Diastolic branch
    diastolic_dense_units = hp.Int('diastolic_dense_units', min_value=16, max_value=64, step=16)
    diastolic_branch = Dense(diastolic_dense_units, activation='relu')(shared)
    diastolic_branch = Dropout(dropout_rate)(diastolic_branch)
    diastolic_output = Dense(3, activation='softmax', name='diastolic')(diastolic_branch)
    
    model = tf.keras.Model(inputs=inputs, outputs=[systolic_output, diastolic_output])
    
    # Create one-vs-rest AUROC metrics for each output.
    systolic_auc = tf.keras.metrics.AUC(multi_label=True, num_labels=3, curve='ROC', name='systolic_auc')
    diastolic_auc = tf.keras.metrics.AUC(multi_label=True, num_labels=3, curve='ROC', name='diastolic_auc')
    
    learning_rate = hp.Choice('learning_rate', [0.001, 0.0005, 0.0001])
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss={'systolic': 'categorical_crossentropy', 'diastolic': 'categorical_crossentropy'},
                  metrics={'systolic': systolic_auc, 'diastolic': diastolic_auc})
    return model

tuner = kt.RandomSearch(
    build_model,
    objective=kt.Objective('val_systolic_systolic_auc', direction='max'),
    max_trials=20,
    executions_per_trial=1,
    directory='lstm_tuner_multioutput',
    project_name='bp_tertile_prediction',
    overwrite=True
)

tuner.search(X_train_lstm, y_train_multi,
             epochs=50,
             batch_size=32,
             validation_data=(X_test_lstm, y_test_multi))

best_model = tuner.get_best_models(num_models=1)[0]
print("🔹 Best Multi-Output Model Hyperparameters:")
print(tuner.get_best_hyperparameters(num_trials=1)[0].values)

#############################################
# 4. Evaluate the Best Model on the Test Set
#############################################
results = best_model.evaluate(X_test_lstm, y_test_multi, verbose=0)
print(f"Test Loss & AUROC (Systolic, Diastolic): {results}")

#############################################
# 5. Print Total Training Time
#############################################
end_time = time.time()
print("Total training time: {:.2f} seconds".format(end_time - start_time))


Trial 20 Complete [00h 01m 51s]
val_systolic_systolic_auc: 0.6725838780403137

Best val_systolic_systolic_auc So Far: 0.682716429233551
Total elapsed time: 00h 32m 05s

🔹 Best Multi-Output Model Hyperparameters:
{'lstm_units': 256, 'dropout_rate': 0.2, 'lstm_units_2': 48, 'attention_variant': 'selfattention', 'shared_dense_units': 32, 'systolic_dense_units': 16, 'diastolic_dense_units': 16, 'learning_rate': 0.001, 'num_heads': 4, 'key_dim': 64}
Test Loss & AUROC (Systolic, Diastolic): [2.113337278366089, 1.0713911056518555, 1.041946291923523, 0.682716429233551, 0.6651015877723694]
Total training time: 1944.48 seconds


In [ ]:
print("🔹 BP Spike Counts Before Resampling:")
print(f"   - Training Set: {sum(y_train)} spikes out of {len(y_train)} samples ({sum(y_train)/len(y_train)*100:.2f}%)")
print(f"   - Test Set: {sum(y_test)} spikes out of {len(y_test)} samples ({sum(y_test)/len(y_test)*100:.2f}%)")

In [6]:
import time
start_time = time.time()

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, Bidirectional, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import ADASYN
import kerastuner as kt

#############################################
# Custom Attention Layers
#############################################
class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight",
                                 shape=(input_shape[-1], 1),
                                 initializer="normal",
                                 trainable=True)
        self.b = self.add_weight(name="att_bias",
                                 shape=(input_shape[1], 1),
                                 initializer="zeros",
                                 trainable=True)
        super(AttentionLayer, self).build(input_shape)
    def call(self, x):
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.backend.softmax(e, axis=1)
        output = x * a
        return tf.keras.backend.sum(output, axis=1)

class MultiHeadAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, num_heads, key_dim, **kwargs):
        super(MultiHeadAttentionLayer, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.key_dim)
    def call(self, inputs):
        attn_output = self.mha(query=inputs, key=inputs, value=inputs)
        return tf.reduce_mean(attn_output, axis=1)

class SelfAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(SelfAttentionLayer, self).__init__(**kwargs)
        self.att = tf.keras.layers.Attention()
    def call(self, x):
        att_out = self.att([x, x])
        return tf.reduce_mean(att_out, axis=1)

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, num_heads, key_dim, ff_dim, rate=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.ff_dim = ff_dim
        self.rate = rate
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
    def build(self, input_shape):
        self.embed_dim = input_shape[-1]
        self.ffn = tf.keras.Sequential([
            Dense(self.ff_dim, activation='relu'),
            Dense(self.embed_dim)
        ])
        super(TransformerBlock, self).build(input_shape)
    def call(self, inputs, training=False):
        attn_output = self.mha(query=inputs, key=inputs, value=inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

#############################################
# Custom Tuner to Print Both AUROC Metrics at Trial End
#############################################
class MyRandomSearch(kt.RandomSearch):
    def on_trial_end(self, trial):
        # Call parent's on_trial_end first.
        super().on_trial_end(trial)
        # Try to access trial history if available.
        if hasattr(trial, 'history') and trial.history is not None:
            history = trial.history.history
            best_val_systolic = max(history.get("val_systolic_systolic_auc", [float('nan')]))
            best_val_diastolic = max(history.get("val_diastolic_diastolic_auc", [float('nan')]))
            print(f"\nTrial {trial.trial_id} complete:")
            print(f"   Best val_systolic_systolic_auc: {best_val_systolic:.4f}")
            print(f"   Best val_diastolic_diastolic_auc: {best_val_diastolic:.4f}")
        else:
            # If history is not available, fallback: print best objective (systolic only)
            print(f"\nTrial {trial.trial_id} complete: trial.history not available. Best objective (systolic) is {trial.score:.4f}")

#############################################
# 1. Load Processed Dataset and Prepare Targets
#############################################
# The processed CSV must include the tertile columns "systolic_tertile" and "diastolic_tertile".
df = pd.read_csv("processed_bp_prediction_data_multiclass.csv")

# Define features (ensure target columns are excluded)
features = [
    'hr_mean_5min', 'hr_min_5min', 'hr_max_5min', 'hr_std_5min',
    'steps_total_5min', 'steps_mean_5min', 'steps_min_5min', 'steps_max_5min', 'steps_std_5min', 'steps_diff_5min',
    'hr_mean_10min', 'hr_min_10min', 'hr_max_10min', 'hr_std_10min',
    'steps_total_10min', 'steps_mean_10min', 'steps_min_10min', 'steps_max_10min', 'steps_std_10min', 'steps_diff_10min',
    'hr_mean_30min', 'hr_min_30min', 'hr_max_30min', 'hr_std_30min',
    'steps_total_30min', 'steps_mean_30min', 'steps_min_30min', 'steps_max_30min', 'steps_std_30min', 'steps_diff_30min',
    'hr_mean_60min', 'hr_min_60min', 'hr_max_60min', 'hr_std_60min',
    'steps_total_60min', 'steps_mean_60min', 'steps_min_60min', 'steps_max_60min', 'steps_std_60min', 'steps_diff_60min',
    'stress_mean', 'stress_min', 'stress_max', 'stress_std',
    'stress_mean_lag_1', 'stress_mean_lag_3', 'stress_mean_lag_5',
    'BP_spike_mean_lag_1', 'BP_spike_mean_lag_3', 'BP_spike_mean_lag_5',
    'hr_mean_5min_lag_1', 'hr_mean_5min_lag_3', 'hr_mean_5min_lag_5',
    'steps_total_10min_lag_1', 'steps_total_10min_lag_3', 'steps_total_10min_lag_5',
    'hr_steps_ratio', 'stress_weighted_hr', 'stress_steps_ratio', 'steps_hr_variability_ratio',
    'hr_mean_rolling_3', 'steps_total_rolling_5', 'hr_std_rolling_3',
    'cumulative_stress_30min', 'cumulative_steps_30min',
    'hour_of_day', 'day_of_week', 'is_working_hours', 'is_weekend',
    'time_since_last_BP_spike'
]

# Our targets: the tertile classifications for systolic and diastolic values
target_systolic = "systolic_tertile"
target_diastolic = "diastolic_tertile"

# Retain only needed columns
df = df[["datetime_local"] + features + [target_systolic, target_diastolic]]
df[features] = df[features].apply(pd.to_numeric, errors='coerce')
df['datetime_local'] = pd.to_datetime(df['datetime_local'])

# Split train and test by time
train_cutoff = df['datetime_local'].min() + pd.Timedelta(days=20)
train_data = df[df['datetime_local'] < train_cutoff]
test_data = df[df['datetime_local'] >= train_cutoff]

X_train = train_data[features].copy()
X_test = test_data[features].copy()

# One-hot encode targets for multi-class classification
y_train_systolic = pd.get_dummies(train_data[target_systolic])
y_train_diastolic = pd.get_dummies(train_data[target_diastolic])
y_test_systolic = pd.get_dummies(test_data[target_systolic])
y_test_diastolic = pd.get_dummies(test_data[target_diastolic])

# Ensure consistent ordering of columns
order = ['low', 'mid', 'high']
y_train_systolic = y_train_systolic[order]
y_train_diastolic = y_train_diastolic[order]
y_test_systolic = y_test_systolic[order]
y_test_diastolic = y_test_diastolic[order]

# Create dictionary targets for the multi-output model
y_train_multi = {'systolic': y_train_systolic.values, 'diastolic': y_train_diastolic.values}
y_test_multi = {'systolic': y_test_systolic.values, 'diastolic': y_test_diastolic.values}

#############################################
# 2. Scaling and Reshaping for LSTM
#############################################
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

#############################################
# 3. Build Multi-Output (Multi-Task) LSTM Model with Keras Tuner
#############################################
def build_model(hp):
    inputs = tf.keras.Input(shape=(X_train_lstm.shape[1], 1))
    
    # Shared LSTM layers
    lstm_units = hp.Int('lstm_units', min_value=64, max_value=256, step=32)
    x = Bidirectional(LSTM(lstm_units, return_sequences=True))(inputs)
    x = BatchNormalization()(x)
    dropout_rate = hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)
    x = Dropout(dropout_rate)(x)
    
    lstm_units_2 = hp.Int('lstm_units_2', min_value=32, max_value=128, step=16)
    x = LSTM(lstm_units_2, return_sequences=True)(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    
    # Attention variant selection
    att_variant = hp.Choice('attention_variant', ['custom', 'multihead', 'selfattention', 'transformer'])
    if att_variant == 'custom':
        att_out = AttentionLayer()(x)
    elif att_variant == 'multihead':
        num_heads = hp.Int('num_heads', min_value=1, max_value=4, step=1)
        key_dim = hp.Int('key_dim', min_value=16, max_value=64, step=16)
        att_out = MultiHeadAttentionLayer(num_heads=num_heads, key_dim=key_dim)(x)
    elif att_variant == 'selfattention':
        att_out = SelfAttentionLayer()(x)
    else:  # transformer variant
        trans_num_heads = hp.Int('trans_num_heads', min_value=1, max_value=4, step=1)
        trans_key_dim = hp.Int('trans_key_dim', min_value=16, max_value=64, step=16)
        ff_dim = hp.Int('ff_dim', min_value=32, max_value=128, step=32)
        transformer_block = TransformerBlock(num_heads=trans_num_heads, key_dim=trans_key_dim, ff_dim=ff_dim, rate=dropout_rate)
        att_out = transformer_block(x)
        att_out = GlobalAveragePooling1D()(att_out)
    
    # Optional shared dense layer
    shared_dense_units = hp.Int('shared_dense_units', min_value=32, max_value=128, step=32)
    shared = Dense(shared_dense_units, activation='relu')(att_out)
    shared = Dropout(dropout_rate)(shared)
    
    # Systolic branch
    systolic_dense_units = hp.Int('systolic_dense_units', min_value=16, max_value=64, step=16)
    systolic_branch = Dense(systolic_dense_units, activation='relu')(shared)
    systolic_branch = Dropout(dropout_rate)(systolic_branch)
    systolic_output = Dense(3, activation='softmax', name='systolic')(systolic_branch)
    
    # Diastolic branch
    diastolic_dense_units = hp.Int('diastolic_dense_units', min_value=16, max_value=64, step=16)
    diastolic_branch = Dense(diastolic_dense_units, activation='relu')(shared)
    diastolic_branch = Dropout(dropout_rate)(diastolic_branch)
    diastolic_output = Dense(3, activation='softmax', name='diastolic')(diastolic_branch)
    
    model = tf.keras.Model(inputs=inputs, outputs=[systolic_output, diastolic_output])
    
    # Create one-vs-rest AUROC metrics for each output.
    systolic_auc = tf.keras.metrics.AUC(multi_label=True, num_labels=3, curve='ROC', name='systolic_auc')
    diastolic_auc = tf.keras.metrics.AUC(multi_label=True, num_labels=3, curve='ROC', name='diastolic_auc')
    
    learning_rate = hp.Choice('learning_rate', [0.001, 0.0005, 0.0001])
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss={'systolic': 'categorical_crossentropy', 'diastolic': 'categorical_crossentropy'},
                  metrics={'systolic': systolic_auc, 'diastolic': diastolic_auc})
    return model

tuner = MyRandomSearch(
    build_model,
    objective=kt.Objective('val_systolic_systolic_auc', direction='max'),
    max_trials=20,
    executions_per_trial=1,
    directory='lstm_tuner_multioutput',
    project_name='bp_tertile_prediction',
    overwrite=True
)

tuner.search(X_train_lstm, y_train_multi,
             epochs=50,
             batch_size=32,
             validation_data=(X_test_lstm, y_test_multi))

best_model = tuner.get_best_models(num_models=1)[0]
print("🔹 Best Multi-Output Model Hyperparameters:")
print(tuner.get_best_hyperparameters(num_trials=1)[0].values)

#############################################
# 4. Evaluate the Best Model on the Test Set
#############################################
results = best_model.evaluate(X_test_lstm, y_test_multi, verbose=0)
metrics = best_model.metrics_names
result_dict = dict(zip(metrics, results))

print("Test Evaluation Metrics:")
print(f"Overall Loss: {result_dict['loss']:.4f}")
print(f"Systolic Loss: {result_dict['systolic_loss']:.4f}")
print(f"Diastolic Loss: {result_dict['diastolic_loss']:.4f}")
print(f"Systolic AUROC: {result_dict['systolic_systolic_auc']:.4f}")
print(f"Diastolic AUROC: {result_dict['diastolic_diastolic_auc']:.4f}")

#############################################
# 5. Print Total Training Time
#############################################
end_time = time.time()
print("Total training time: {:.2f} seconds".format(end_time - start_time))


Trial 20 Complete [00h 02m 03s]
val_systolic_systolic_auc: 0.6322672963142395

Best val_systolic_systolic_auc So Far: 0.6723673343658447
Total elapsed time: 00h 28m 05s

Trial 19 complete: trial.history not available. Best objective (systolic) is 0.6323
🔹 Best Multi-Output Model Hyperparameters:
{'lstm_units': 160, 'dropout_rate': 0.2, 'lstm_units_2': 32, 'attention_variant': 'selfattention', 'shared_dense_units': 96, 'systolic_dense_units': 32, 'diastolic_dense_units': 64, 'learning_rate': 0.0001, 'num_heads': 3, 'key_dim': 16}
Test Evaluation Metrics:
Overall Loss: 2.1921
Systolic Loss: 1.0933
Diastolic Loss: 1.0988
Systolic AUROC: 0.6724
Diastolic AUROC: 0.6278
Total training time: 1696.08 seconds
